In [2]:
# Import various python packages

import iris
import iris.plot as iplt
import iris.quickplot as qplt
import iris.coord_categorisation as icc
from iris.time import PartialDateTime
import iris.util as iutil

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, FixedFormatter
import cartopy.crs as ccrs

import os
import warnings 

import cartopy.io.shapereader as shpreader

import cartopy.feature as cfeature
import matplotlib.path as mpath
from matplotlib.colors import LinearSegmentedColormap, Normalize
from matplotlib.colors import TwoSlopeNorm
from matplotlib.lines import Line2D

from scipy.optimize import minimize, LinearConstraint, Bounds

from pathlib import Path
import requests

warnings.filterwarnings('ignore') 

In [3]:
import sys
sys.path.append('..')

from Cube_Functions import *
from Plot_Functions import *
from Climate_Functions import *
from Stats_Functions import *
print('Finished loading libraries and functions')

#clean up namespace
del sys

Finished loading libraries and functions


In [ ]:
root_path = '/gws/nopw/j04/pmip4_vol1/public/matt/data/'

projection = 'ssp245'

var = 'siconc'

factor = 1e14

RMSE: 0.8760404489546725
Weights:
  CNRM-CM6-1      0.3520
  NESM3           0.2475
  EC-Earth3       0.1758
  HadGEM3-GC31-LL 0.1692
  NorESM2-LM      0.0555
  NorESM1-F       0.0000
  CESM2           0.0000
  ACCESS-ESM1-5   0.0000
  IPSL-CM6A-LR    0.0000

In [5]:
weights_dict = { 'CNRM-CM6-1': 0.3520,
                 'NESM3': 0.2475,
                 'EC-Earth3': 0.1758,
                 'HadGEM3-GC31-LL': 0.1692,
                 'NorESM2-LM': 0.0555}

In [10]:
models = list(weights_dict.keys())

models_with_files = []
#for every folder in the root path
for folder in models:
    #if the folder is a directory and contains the experiment
    print(folder)
    if os.path.isdir(os.path.join(root_path, folder, projection)):
        #print(f'Found {experiment} folder: {folder}')
        #check if there's a file like *_siconc_*.nc in the experiment folder
        exp_path = os.path.join(root_path, folder, projection)
        exp_matches = list(Path(exp_path).glob("siconc_*.nc"))
        if exp_matches:
            print(f"Found {projection}:", len(exp_matches), " files")
            print(exp_matches)
            models_with_files.append(folder)
        else:
            print("No matching files")

print(f"Models with files: {models_with_files}")

CNRM-CM6-1
Found ssp245: 1  files
[PosixPath('/gws/nopw/j04/pmip4_vol1/public/matt/data/CNRM-CM6-1/ssp245/siconc_SImon_CNRM-CM6-1_ssp245_r1i1p1f2_gn_201501-210012.nc')]
NESM3
Found ssp245: 1  files
[PosixPath('/gws/nopw/j04/pmip4_vol1/public/matt/data/NESM3/ssp245/siconc_SImon_NESM3_ssp245_r1i1p1f1_gn_201501-210012.nc')]
EC-Earth3
Found ssp245: 1  files
[PosixPath('/gws/nopw/j04/pmip4_vol1/public/matt/data/EC-Earth3/ssp245/siconc_SImon_EC-Earth3_ssp245_r1i1p1f1_gn_201501-210012.nc')]
HadGEM3-GC31-LL
Found ssp245: 1  files
[PosixPath('/gws/nopw/j04/pmip4_vol1/public/matt/data/HadGEM3-GC31-LL/ssp245/siconc_SImon_HadGEM3-GC31-LL_ssp245_r1i1p1f3_gn_201501-210012.nc')]
NorESM2-LM
Found ssp245: 1  files
[PosixPath('/gws/nopw/j04/pmip4_vol1/public/matt/data/NorESM2-LM/ssp245/siconc_SImon_NorESM2-LM_ssp245_r1i1p1f1_gn_201501-210012.nc')]
Models with files: ['CNRM-CM6-1', 'NESM3', 'EC-Earth3', 'HadGEM3-GC31-LL', 'NorESM2-LM']


In [16]:
lat_limit = 60
con = iris.Constraint(latitude=lambda lat: lat_limit <= lat <= 90)

time_con = iris.Constraint(year=lambda y:np.logical_and(y>=2035,y<2045))  #time_constraint. Not used anywhere yet.

ocean_shp_reader = shpreader.Reader(shpreader.natural_earth(resolution="110m", category="physical", name="ocean"))
ocean_list = []
for ocean in ocean_shp_reader.records():
    ocean_list.append(ocean.geometry)
ocean_shp = ocean_list[1]

shape = ocean_shp

shape = None

In [ ]:
#open all the siconc files for the models with files
siconc_monthly_dict = {}
for model in models:
    print('model: ', model)
    path = create_path(model, projection, var, root_path)
    path = path.replace('regrid','*')
    cube = iris.load_cube(path, var)
    cube_with_coords = add_coords(cube)

    region_constraint = iris.Constraint(latitude=lambda lat: lat >= lat_limit)
    region_cube = cube_with_coords.extract(region_constraint)
    monthly_data = get_monthly_data(cube, 'SUM') / factor
    siconc_monthly_dict[model] = monthly_data


model:  CNRM-CM6-1


CoordinateMultiDimError: Cannot apply constraints to multidimensional coordinates

In [18]:
    cube

Sea Ice Area Fraction (%),time,--,--
Shape,1032,294,362
Dimension coordinates,,,
time,x,-,-
Auxiliary coordinates,,,
day_of_year,x,-,-
month,x,-,-
month_number,x,-,-
year,x,-,-
latitude,-,x,x
longitude,-,x,x
